In [1]:
#Import the dependencies
from bs4 import BeautifulSoup
# import pandas as pd
import requests
# import urllib.request
# import time

# Creating custom scraping function for Poynter

In [2]:
#Create lists to store the scraped data
authors = []
dates = []
statements = []
sources = []
targets = []

In [3]:
#Create a function to scrape the site
def scrape_poynter(page_number):
    page_num = str(page_number) #Convert the page number to a string
    URL = 'https://www.poynter.org/ifcn-covid-19-misinformation/page/'+page_num #append the page number to complete the URL
    webpage = requests.get(URL)  #Make a request to the website
    #time.sleep(3)
    soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
    #Get the tags and it's class
    statement_entry =  soup.find_all('p',attrs={'class':'entry-content__text'})  #Get the tag and it's class
    statement_quote = soup.find_all('h2', attrs={'class':'entry-title'}) #Get the tag and it's class
#     statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
#     target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class
    #loop through the footer class m-statement__footer to get the date and author
    for i in statement_entry:
        link1 = i.text.strip()
#         name_and_date = link1.split()
#         first_name = name_and_date[1]
#         last_name = name_and_date[2]
#         full_name = first_name+' '+last_name
#         month = name_and_date[4]
#         day = name_and_date[5]
#         year = name_and_date[6]
#         date = month+' '+day+' '+year
#         dates.append(date)
        authors.append(link1)
    #Loop through the div m-statement__quote to get the link
#     for i in statement_quote:
#     link2 = i.find_all('a')
#     statements.append(link2[0].text.strip())
#     #Loop through the div m-statement__meta to get the source
#     for i in statement_meta:
#     link3 = i.find_all('a') #Source
#     source_text = link3[0].text.strip()
#     sources.append(source_text)
#     #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
#     for i in target:
#     fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
#     targets.append(fact)

In [5]:
scrape_poynter(1)

In [6]:
authors

[]

In [4]:

# 'https://www.poynter.org/ifcn-covid-19-misinformation/page/2/?__cf_chl_jschl_tk__=pmd_imssgWM9ppiSUtNP9XYEFo.bLVHeJDucbz9ZXy03xxk-1630049234-0-gqNtZGzNAfujcnBszQr9'
URL = 'https://www.poynter.org/ifcn-covid-19-misinformation/' #append the page number to complete the URL
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
webpage = requests.get(URL, headers=headers)  #Make a request to the website
soup = BeautifulSoup(webpage.content, "html.parser") #Parse the text from the website

soup.find_all('p',attrs={'class':'entry-content__text'})

[]

In [40]:
URL = 'https://www.poynter.org/ifcn-covid-19-misinformation/?__cf_chl_jschl_tk__=pmd_imssgWM9ppiSUtNP9XYEFo.bLVHeJDucbz9ZXy03xxk-1630049234-0-gqNtZGzNAfujcnBszQr9'

In [45]:
URL = 'https://www.poynter.org/ifcn-covid-19-misinformation/'

In [46]:
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
#           'referer': 'https://www.poynter.org/ifcn-covid-19-misinformation/'}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


result = requests.head(URL, headers=headers, allow_redirects=True)
result.raise_for_status()

HTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://www.poynter.org/ifcn-covid-19-misinformation/

In [31]:
requests.get('https://www.politifact.com/factchecks/list/')

<Response [200]>

In [36]:
requests.get('https://www.poynter.org/ifcn-covid-19-misinformation/', headers = headers)

<Response [503]>

In [22]:
#Model found online using scrapy - not tested yet

import scrapy


class PoynterFakenewsSpider(scrapy.Spider):
    name = 'Poynter_FakeNews'
    allowed_domains = ['poynter.org']
    start_urls = ['https://www.poynter.org/ifcn-covid-19-misinformation//']

    custom_settings={ 'FEED_URI': "crawlPoynter_%(time)s.csv", 'FEED_FORMAT': 'csv'} 

    def parse(self, response):

        for article in response.xpath('//article'):
            Title = article.xpath('.//h2[@class="entry-title"]/a/text()').get()
            Date = article.xpath('.//p[@class="entry-content__text"]/strong/text()').get()
            ReadMore_link = article.xpath('.//a[@class="button entry-content__button entry-content__button--smaller"]/@href').get()

            yield scrapy.Request(
                url=response.urljoin(ReadMore_link), 
                callback=self.parse_article_details,
                cb_kwargs={
                    'article_title': Title,
                    'article_date': Date,
                }
            )
        next_page = response.xpath('//a[@class="next page-numbers"]/@href').extract_first()
        if next_page: 
            yield scrapy.Request(response.urljoin(next_page), callback=self.parse)

    def parse_article_details(self, response, article_title, article_date):
        Source = response.xpath('//p[@class="entry-content__text entry-content__text--smaller"]/text()').extract_first()
        scraped_info = {
            'page':response.url,
            'Title': article_title, 
            'Date': article_date,
            'Source': Source,
        }
        yield scraped_info

ModuleNotFoundError: No module named 'scrapy'